In [108]:
from qiskit_nature.drivers.second_quantization import HDF5Driver
driver_reduced = HDF5Driver("./PSPCz_reduced.hdf5")
properties = driver_reduced.run()


In [109]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
es_problem = ElectronicStructureProblem(driver_reduced)
second_q_op = es_problem.second_q_ops()


In [110]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper, BravyiKitaevMapper
qubit_converter = QubitConverter(JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
print(qubit_op)

-0.45781773131305903 * IIII
- 0.009666607989543467 * ZIII
+ 0.12689900731767084 * IZII
+ 0.030293077447785 * ZZII
- 0.009666607989543479 * IIZI
+ 0.03732964036584735 * ZIZI
+ 0.034223590264106186 * IZZI
+ 0.12689900731767084 * IIIZ
+ 0.034223590264106186 * ZIIZ
+ 0.05698995686717464 * IZIZ
+ 0.030293077447785 * IIZZ
+ 0.00014809461815615455 * XXII
+ 0.00014809461815615455 * YYII
- 7.872869677230731e-05 * XXZI
- 7.872869677230731e-05 * YYZI
+ 6.938452207544002e-05 * XXIZ
+ 6.938452207544002e-05 * YYIZ
+ 0.00014809461815615455 * IIXX
- 7.872869677230731e-05 * ZIXX
+ 6.938452207544002e-05 * IZXX
+ 0.00014809461815615455 * IIYY
- 7.872869677230731e-05 * ZIYY
+ 6.938452207544002e-05 * IZYY
+ 0.003930512816321183 * XXXX
+ 0.003930512816321183 * YYXX
+ 0.003930512816321183 * XXYY
+ 0.003930512816321183 * YYYY


In [111]:
from qiskit_nature.circuit.library import HartreeFock
init_state = HartreeFock(4, (1,1), qubit_converter)
init_state.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────

In [112]:
from qiskit.circuit.library import EfficientSU2, TwoLocal, NLocal, PauliTwoDesign
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

ansatz = EfficientSU2(num_qubits = 4,
                     su2_gates = ['h', 'ry'], 
                    entanglement = 'full',
                    reps = 2,
                    initial_state = init_state)
ansatz.decompose().draw()

┌───┐   ┌───┐    ┌──────────┐                    ┌───┐┌──────────┐»
q_0: ┤ X ├───┤ H ├────┤ Ry(θ[0]) ├──■────■─────────■──┤ H ├┤ Ry(θ[4]) ├»
     ├───┤┌──┴───┴───┐└──────────┘┌─┴─┐  │         │  └───┘└──┬───┬───┘»
q_1: ┤ H ├┤ Ry(θ[1]) ├────────────┤ X ├──┼────■────┼────■─────┤ H ├────»
     ├───┤└──┬───┬───┘┌──────────┐└───┘┌─┴─┐┌─┴─┐  │    │     └───┘    »
q_2: ┤ X ├───┤ H ├────┤ Ry(θ[2]) ├─────┤ X ├┤ X ├──┼────┼───────■──────»
     ├───┤┌──┴───┴───┐└──────────┘     └───┘└───┘┌─┴─┐┌─┴─┐   ┌─┴─┐    »
q_3: ┤ H ├┤ Ry(θ[3]) ├───────────────────────────┤ X ├┤ X ├───┤ X ├────»
     └───┘└──────────┘                           └───┘└───┘   └───┘    »
«                                            ┌───┐┌──────────┐            »
«q_0: ─────────────────■────────■─────────■──┤ H ├┤ Ry(θ[8]) ├────────────»
«     ┌──────────┐   ┌─┴─┐      │         │  └───┘└──┬───┬───┘┌──────────┐»
«q_1: ┤ Ry(θ[5]) ├───┤ X ├──────┼────■────┼────■─────┤ H ├────┤ Ry(θ[9]) ├»
«     └──┬───┬───┘┌──┴───┴───┐┌─┴─┐┌─┴─┐  │    │     └───┘    └──┬───┬───┘»
«q_2: ───┤ H ├────┤ Ry(θ[6]) ├┤ X ├┤ X ├──┼────┼───────■─────────┤ H ├────»
«        ├───┤    ├──────────┤└───┘└───┘┌─┴─┐┌─┴─┐   ┌─┴─┐       ├───┤    »
«q_3: ───┤ H ├────┤ Ry(θ[7]) ├──────────┤ X ├┤ X ├───┤ X ├───────┤ H ├────»
«        └───┘    └──────────┘          └───┘└───┘   └───┘       └───┘    »
«                  
«q_0: ─────────────
«                  
«q_1: ─────────────
«     ┌───────────┐
«q_2: ┤ Ry(θ[10]) ├
«     ├───────────┤
«q_3: ┤ Ry(θ[11]) ├
«     └───────────┘

In [113]:
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
backend = StatevectorSimulator()
optimizer = COBYLA()

In [114]:
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import VQEUCCFactory, GroundStateEigensolver
from jupyterplot import ProgressPlot
import numpy as np
error_threshold = 10
np.random.seed(5)
initial_point = np.random.random(ansatz.num_parameters)

In [115]:
vqe = VQE(ansatz = ansatz,
         optimizer = optimizer,
         initial_point = initial_point,
         quantum_instance = backend)

vqe_ground_state_solver = GroundStateEigensolver(qubit_converter,
                                                vqe)
vqe_results = vqe_ground_state_solver.solve(es_problem)
print(vqe_results)


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -4043.631987584531
  - computed part:      -0.765665024439
  - ActiveSpaceTransformer extracted energy part: -4042.866322560092
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000


In [116]:
from qiskit_nature.algorithms import QEOM
qeom_excited_state_solver = QEOM(vqe_ground_state_solver, 'sd')
qeom_results = qeom_excited_state_solver.solve(es_problem)

print(qeom_results)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -4043.631987584531
  - computed part:      -0.765665024439
  - ActiveSpaceTransformer extracted energy part: -4042.866322560092
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -4043.426917298302
* Electronic excited state energy (Hartree): -4043.426917298302
> Total excited state energy (Hartree): -4043.426917298302
  2: 
* Electronic excited state energy (Hartree): -4043.395388322966
* Electronic excited state energy (Hartree): -4043.395388322966
> Total excited state energy (Hartree): -4043.395388322966
  3: 
* Electronic excited state energy (Hartree): -4043.085721824136
* Electronic excited state energy (Hartree): -4043.085721824136
> Total excited state energy (Hartree): -4043.085721824136
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000


In [117]:
bandgap = qeom_results.computed_energies[1] - qeom_results.computed_energies[0]
bandgap

0.20507028622962853

In [118]:
vqe0 = vqe.get_optimal_circuit()

In [119]:
from qiskit.opflow import I, X, Y, Z
print(I, X, Y, Z)
H_op = Z ^ I ^ X ^ Y
print(H_op)
qubit_converter = QubitConverter(JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
H_op = qubit_op
print(H_op)

I X Y Z
ZIXY
-0.45781773131305903 * IIII
- 0.009666607989543467 * ZIII
+ 0.12689900731767084 * IZII
+ 0.030293077447785 * ZZII
- 0.009666607989543479 * IIZI
+ 0.03732964036584735 * ZIZI
+ 0.034223590264106186 * IZZI
+ 0.12689900731767084 * IIIZ
+ 0.034223590264106186 * ZIIZ
+ 0.05698995686717464 * IZIZ
+ 0.030293077447785 * IIZZ
+ 0.00014809461815615455 * XXII
+ 0.00014809461815615455 * YYII
- 7.872869677230731e-05 * XXZI
- 7.872869677230731e-05 * YYZI
+ 6.938452207544002e-05 * XXIZ
+ 6.938452207544002e-05 * YYIZ
+ 0.00014809461815615455 * IIXX
- 7.872869677230731e-05 * ZIXX
+ 6.938452207544002e-05 * IZXX
+ 0.00014809461815615455 * IIYY
- 7.872869677230731e-05 * ZIYY
+ 6.938452207544002e-05 * IZYY
+ 0.003930512816321183 * XXXX
+ 0.003930512816321183 * YYXX
+ 0.003930512816321183 * XXYY
+ 0.003930512816321183 * YYYY


In [120]:
ansatz = EfficientSU2(num_qubits = 4,
                     su2_gates = ['h', 'ry'], 
                    entanglement = 'full',
                    reps = 2,
                    initial_state = init_state)

optimizer = optimizer
initial_point = np.random.random(ansatz.num_parameters)
backend = StatevectorSimulator()

vqe = VQE(ansatz = ansatz,
         optimizer = optimizer,
         initial_point = initial_point,
         quantum_instance = backend)


In [121]:
result = vqe.compute_minimum_eigenvalue(operator=H_op)
# I just realized how impossible H_op + overlap is
# vqe solves <ansatz | H | ansatz>
# vqd solves <ansatz | H | ansatz> + |<ansatz | prev_ansatz>|^2
# <ansatz | Penalty_op | ansatz>
# Penalty_op = prev_ansatz*[I + Z] ^ n * prev_ansatz / 2^n
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 1000,
    'eigenstate': array([ 3.12089778e-06+2.10595821e-19j,  1.51174401e-04-4.72068614e-17j,
        6.18394864e-06-7.55405928e-20j, -4.27267088e-08-5.00445328e-17j,
        2.27987743e-04-6.06739419e-17j,  9.99972091e-01+7.24030275e-16j,
        1.95765032e-05-2.67992627e-20j, -2.62410227e-04+1.30806044e-16j,
        7.15364899e-03+4.77045389e-18j, -3.55029889e-06-5.26076717e-17j,
       -2.10158123e-03-2.42611241e-18j,  8.78081158e-07+1.57407772e-19j,
        3.40209725e-05-1.66734866e-19j, -2.82937896e-04+3.11578667e-17j,
       -1.00190647e-05-3.35283006e-19j,  1.28824509e-07-4.44409188e-20j]),
    'eigenvalue': (-0.7656965875992804+0j),
    'optimal_parameters': {   ParameterVectorElement(θ[7]): 8.269378641811124e-05,
                              ParameterVectorElement(θ[11]): 1.5593630759698853,
                              ParameterVectorElement(θ[9]): 1.2847958198590805,
                              Paramet

In [122]:
n = 4
print(n)
#zero_proj = (1 / 2**n) * (I + Z) ** n
zero_proj = (1 / 2**n) * (I + Z) ^ (I + Z) ^ (I + Z) ^ (I + Z)
print(zero_proj)

4
0.0625 * IIII
+ 0.0625 * IIIZ
+ 0.0625 * IIZI
+ 0.0625 * IIZZ
+ 0.0625 * IZII
+ 0.0625 * IZIZ
+ 0.0625 * IZZI
+ 0.0625 * IZZZ
+ 0.0625 * ZIII
+ 0.0625 * ZIIZ
+ 0.0625 * ZIZI
+ 0.0625 * ZIZZ
+ 0.0625 * ZZII
+ 0.0625 * ZZIZ
+ 0.0625 * ZZZI
+ 0.0625 * ZZZZ


In [123]:
from qiskit.opflow istmport CircuitOp
opt_circ = vqe.get_optimal_circuit()
test_state = CircuitOp(opt_circ)

SyntaxError: invalid syntax (2777693715.py, line 1)

In [124]:
print(test_state.adjoint())
    

IndentationError: expected an indented block (1673459248.py, line 3)

In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.parametervector import ParameterVector
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.opflow import CircuitStateFn, StateFn
from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki

H_op = (0.5 * Z) 
ansatz = QuantumCircuit(1)
nu = ParameterVector('nu', 2)
ansatz.h(0)
ansatz.rz(nu[0], 0)
ansatz.ry(nu[1], 0)

opt = SLSQP(maxiter=1000)
qi = StatevectorSimulator()
vqe = VQE(ansatz, optimizer=opt, quantum_instance=qi)
result = vqe.compute_minimum_eigenvalue(operator=H_op)

opt_circ = vqe.get_optimal_circuit() # seems to be deprecated - what is the new implementation?

test_state = CircuitOp(opt_circ)
zero_proj = (0.5 * (I+Z)) 
print(H_op)


NameError: name 'SLSQP' is not defined

In [128]:
circuit_ansatz = CircuitStateFn(ansatz)
print(circuit_ansatz)

print(StateFn(H_op).adjoint())
H_adj = StateFn(H_op).adjoint()
ansatz_and_H = H_adj @ circuit_ansatz
print("Pauli")
print(PauliExpectation().convert(ansatz_and_H))
print()
print(ansatz_and_H)
sampler = CircuitSampler(backend=backend)
init_points = list(range(2))
binded = ansatz_and_H.bind_parameters({nu: init_points})
sampled = sampler.convert(binded)
print(sampled.eval())
# adj = Operator(test_state.adjoint())
# pre = Operator(test_state)
# print(adj)
# H_op = Operator(H_op).compose(pre)
# adj.compose(H_op)
# # + adj @ zero_proj @ pre
result_ex = vqe.compute_minimum_eigenvalue(operator=H_op)
#print(result_ex)

CircuitStateFn(
     ┌───┐┌───────────┐┌───────────┐
q_0: ┤ H ├┤ Rz(nu[0]) ├┤ Ry(nu[1]) ├
     └───┘└───────────┘└───────────┘
)
OperatorMeasurement(0.5 * Z)
Pauli
ComposedOp([
  OperatorMeasurement(0.5 * Z),
  CircuitStateFn(
       ┌───┐┌───────────┐┌───────────┐
  q_0: ┤ H ├┤ Rz(nu[0]) ├┤ Ry(nu[1]) ├
       └───┘└───────────┘└───────────┘
  )
])

ComposedOp([
  OperatorMeasurement(0.5 * Z),
  CircuitStateFn(
       ┌───┐┌───────────┐┌───────────┐
  q_0: ┤ H ├┤ Rz(nu[0]) ├┤ Ry(nu[1]) ├
       └───┘└───────────┘└───────────┘
  )
])
(-0.4207354924039482+0j)


In [154]:
# initialize toy model xD
H_op = (0.5 * Z) 
ansatz = QuantumCircuit(1)
nu = ParameterVector('nu', 2)
ansatz.h(0)
ansatz.rz(nu[0], 0)
ansatz.ry(nu[1], 0)

# Convert stuff into OpFlow
H_meas = StateFn(H_op, is_measurement=True) # is_measurement
circuit_ansatz = CircuitStateFn(ansatz) # Circuit?
H_and_ansatz = H_meas @ circuit_ansatz # whatever this is

# had to figure to bind parameters here
sampler = CircuitSampler(backend=backend)
init_points = list(range(2))
binded = H_and_ansatz.bind_parameters({nu: init_points}) #works?
sampled = sampler.convert(binded)

print(sampled.eval())


(-0.4207354924039482+0j)


In [160]:
opt_circ = vqe.get_optimal_circuit() 
zero_proj = (0.5 * (I+Z)) # like H_op

zero_proj_meas = StateFn(zero_proj, is_measurement=True) 
Penalty_ansatz = CircuitStateFn(ansatz + opt_circ.inverse()) # like an ansatz
Penalty_and_ansatz = zero_proj_meas @ Penalty_ansatz

H_tot = H_and_ansatz + Penalty_and_ansatz

sampler = CircuitSampler(backend=backend)
init_points = list(range(2))
binded = H_tot.bind_parameters({nu: init_points}) #works?
sampled = sampler.convert(binded)

print(sampled.eval())

(0.5001182291522088+0j)
